In [9]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import unidecode
import re
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem.snowball import SnowballStemmer
import datetime
import time

------------------
### Preparación del dataset

In [10]:
data = pd.read_csv("../Data procesada/data_audio.csv")
data.drop(columns=['Unnamed: 0'],inplace=True)
data.head()

,folder,path_audio,file,date,text,start,end
0,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49,hola hola muy buenas tardes bienvenidos al dia...,2020-08-14 17:56:14,2020-08-14 17:56:33
1,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49,tenemos que hablar de esta definición por part...,2020-08-14 17:56:34,2020-08-14 17:57:19
2,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49,algún punto empieza a reconocer también y si u...,2020-08-14 17:57:19,2020-08-14 17:58:04
3,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49,hay una circulación realmente muy alta como es...,2020-08-14 17:58:04,2020-08-14 17:58:50
4,C5N,../Data/Audio/C5N/C5N2020-08-14-17-55-49.wav,C5N2020-08-14-17-55-49.wav,2020-08-14 17:55:49,en esta jornada tan importante hablamos de esp...,2020-08-14 17:58:50,2020-08-14 17:59:35


------------
### Ventana temporal

In [11]:
data['start'] = data['start'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
data['end'] = data['end'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [12]:
delta = 20
td = datetime.timedelta(minutes=delta)

In [13]:
registrosAudio =  pd.DataFrame(columns=['folder', 'file','deltaStart', 'deltaEnd','deltaStep','text'])

for file in data['file'].unique():
    
    maskFile =  data['file'] == file
    
    folder = data.loc[maskFile, 'folder'].unique()
    
    deltaStart = data.loc[maskFile, 'start'].min()
    deltaEnd = deltaStart + td
    
    endFile = data.loc[maskFile, 'end'].max()
    deltaStep=1
    while deltaStart <= endFile:
        
        
        maskDelta = (data['start'] >= deltaStart) & (data['end'] < deltaEnd)
        
        data.loc[maskDelta & maskFile,'deltaStart'] = deltaStart
        
        data.loc[maskDelta & maskFile,'deltaEnd'] = deltaEnd

        text = data.loc[maskDelta & maskFile,'text'].unique()
        textDelta = ','.join(text)
        newInput=pd.DataFrame([textDelta], columns=['text'])
        newInput['file'] = file
        newInput['folder'] = folder
        newInput['deltaStart'] = deltaStart
        newInput['deltaEnd'] = deltaEnd
        newInput['deltaStep']=deltaStep
        newInput.reset_index(inplace=True,drop=True)
        
        registrosAudio = registrosAudio.append(newInput)

        deltaStart = deltaStart + td
        deltaEnd = deltaStart + td
        deltaStep=deltaStep+1
registrosAudio['file'] = registrosAudio['file'].apply(lambda x: x.split('.')[0]) 

len_mean = registrosAudio['text'].apply(lambda x: len(x)).mean()

mask_filter = registrosAudio['text'].apply(lambda x: len(x)>(len_mean*0.7))


registrosAudio=registrosAudio.loc[mask_filter,:]
registrosAudio.reset_index(drop=True,inplace=True)
print(registrosAudio.shape)
registrosAudio.head()

(12, 6)


,folder,file,deltaStart,deltaEnd,deltaStep,text
0,C5N,C5N2020-08-14-17-55-49,2020-08-14 17:56:14,2020-08-14 18:16:14,1,hola hola muy buenas tardes bienvenidos al dia...
1,C5N,C5N2020-08-14-17-55-49,2020-08-14 18:16:14,2020-08-14 18:36:14,2,ellos saben que están pasando y se obligan a s...
2,C5N,C5N2020-08-14-17-55-49,2020-08-14 18:36:14,2020-08-14 18:56:14,3,de las camas respaldan por lo general faltan d...
3,C5N,C5N2020-08-14-17-55-49,2020-08-14 18:56:14,2020-08-14 19:16:14,4,son altas área cama de star wars como creen qu...
4,C5N,C5N2020-08-14-17-55-49,2020-08-14 19:16:14,2020-08-14 19:36:14,5,se a manifestarse está claro que nadie quiere ...


In [14]:
aux = pd.Series(registrosAudio['file'].apply(lambda x: x.replace(".wav","").split("-")[-6:]))
aux = aux.apply(lambda x: [int(a) if len(a)<=4 else int(a[-4:]) for a in x])
registrosAudio['date'] = aux.apply(lambda x: datetime.datetime(x[0], x[1], x[2], hour=x[3], minute=x[4], second=x[5]))
file = registrosAudio['folder'] +" "+ registrosAudio['date'].apply(lambda x: str(x.month)+"-"+str(x.day))+" Step "+registrosAudio['deltaStep'].apply(lambda x: str(x))
file.reset_index(inplace=True,drop=True)
file.head()

0    C5N 8-14 Step 1
1    C5N 8-14 Step 2
2    C5N 8-14 Step 3
3    C5N 8-14 Step 4
4    C5N 8-14 Step 5
dtype: object

In [15]:
textos = registrosAudio.loc[:,'text']

textos_limpios=[];
for t in textos:
    t_lower_no_accents=unidecode.unidecode(t.lower()); # sacamos acentos y llevamos a minuscula
    t_lower_no_accents_no_punkt=re.sub(r'([^\s\w]|_)+','',t_lower_no_accents); # quitamos signos de puntuacion usando una regex que reemplaza todo lo q no sean espacios o palabras por un string vacio
    textos_limpios.append(t_lower_no_accents_no_punkt)

----------
### Stopwords

In [16]:
stopwords_sp_1 = pd.DataFrame(stopwords.words('spanish'),columns=["stopwords"])
stopwords_sp_2 = pd.read_csv("../Varios/stopwords.txt",header=None,names=["stopwords"]) #https://countwordsfree.com/stopwords/spanish/txt
print("stopwords_sp_1:",stopwords_sp_1.shape[0])
print("stopwords_sp_2:",stopwords_sp_2.shape[0])

stopwords_sp_1: 313
stopwords_sp_2: 443


In [17]:
stopwords_sp_3 = stopwords_sp_2.append(stopwords_sp_1)

In [18]:
stopwords_sp = []
for word in stopwords_sp_3.values:
    word_clean =  unidecode.unidecode(str(word[0]).lower())
    if word_clean not in stopwords_sp:
        stopwords_sp.append(word_clean)

In [19]:
print("Conjunto final de stopwords:",len(stopwords_sp))

Conjunto final de stopwords: 591


----------
### Stemming

Aplicamos stemming a las stopwords

In [20]:
def stemfraseesp_for_stopwords(frase):    
    token_words=word_tokenize(frase)
    stem_sentence=[]    
    spanishStemmer=SnowballStemmer("spanish")
    for word in token_words:
        stem_sentence.append(spanishStemmer.stem(word))
    return "".join(stem_sentence)

In [21]:
stopwords_sp_stemming = []
for word in stopwords_sp:
    stopword_stem = stemfraseesp_for_stopwords(word)
    if stopword_stem not in stopwords_sp_stemming:
        stopwords_sp_stemming.append(stopword_stem)

Creamos una tabla que nos permita conservar cuál es la palabra origina de cada stemming para que luego de contar las palabras podamos transformarla en su forma original

In [22]:
texto_completo = "".join(registrosAudio['text'])
texto_completo=unidecode.unidecode(texto_completo.lower()); # sacamos acentos y llevamos a minuscula
texto_completo=re.sub(r'([^\s\w]|_)+','',texto_completo); # quitamos signos de puntuacion usando una regex que reemplaza todo lo q no sean espacios o palabras por un string vacio

In [23]:
token_words=word_tokenize(texto_completo)
stem_sentence=[]    
token_words_clean=[]
spanishStemmer=SnowballStemmer("spanish")
for word in token_words:
    if len(word)>2:
        word_stem = spanishStemmer.stem(word)
        if word_stem not in stopwords_sp_stemming:
            stem_sentence.append(word_stem)
            token_words_clean.append(word)

In [24]:
stem_match = pd.DataFrame([token_words_clean,stem_sentence],index=['original','stem']).T
stem_match =  pd.DataFrame(stem_match.groupby('stem')['original'].value_counts())
stem_match.columns = ['count']
stem_match.reset_index(inplace=True)

In [25]:
stem_match = stem_match.sort_values(by='count', ascending=False)
stem_match.drop_duplicates(subset=['stem'],keep='first', inplace=True)
stem_match.reset_index(drop=True,inplace=True)
print("Tabla para matcheo")
print(stem_match.shape)
stem_match.head()

Tabla para matcheo
(2924, 3)


,stem,original,count
0,cuarenten,cuarentena,104
1,argentin,argentina,74
2,ciud,ciudad,71
3,president,presidente,69
4,gobiern,gobierno,67


Realizamos el stemming de los textos que utilizarmos para procesar

In [26]:
def stemfraseesp(frase):    
    token_words=word_tokenize(frase)
    stem_sentence=[]    
    token_words_clean=[]
    spanishStemmer=SnowballStemmer("spanish")
    for word in token_words:
        if len(word)>2:
            word_stem = spanishStemmer.stem(word)
            if word_stem not in stopwords_sp_stemming:
                stem_sentence.append(word_stem)
                stem_sentence.append(" ")
    return "".join(stem_sentence)

In [27]:
textos_final = []
for t in textos_limpios:
    textos_final.append(stemfraseesp(t))

In [28]:
print("Textos final con stemming.\nEjemplo:")
textos_final[0][0:500]

Textos final con stemming.
Ejemplo:


'hol hol tard bienven diari pantall jorn muchisim informacion minut const contact conductor program pabl empez repas titular dia diri lugarten habl definicion president nacion albert fernandez cuarenten analiz fras muchisim president respond sector socied polit med levant ded acus cuarenten sirv element cas impact cas revest pens detall definicion polit detr vei reci clar gobiern enalgun punt empiez reconoc sal call ves activ socied punt establec limitacion gan vid context rig cuestion sanitari h'

------------------------
### CountVectorizer

In [29]:
vectorizer=CountVectorizer(stop_words=stopwords_sp,lowercase=True,ngram_range=(1,1), max_df=0.7); #revisar si tenemos que poner stopwords acá, creo que no
vectorizer.fit(textos_final);

Matcheamos los stemming con su nombre original. Luego usaremos la variable feature_names para nombrar las columnas

In [30]:
feature_names = pd.DataFrame()
if len(vectorizer.get_feature_names()[0].split(' '))==2:
    aux = pd.Series(vectorizer.get_feature_names())
    aux = aux.str.split(' ',expand=True)
    aux.columns= ['word1','word2']
    aux = aux.merge(stem_match,how='left', left_on='word1',right_on='stem')
    aux = aux.merge(stem_match,how='left', left_on='word2',right_on='stem',suffixes=('_1','_2'))
    features_names= aux['original_1']+" "+aux['original_2']
else:
    aux = pd.DataFrame(vectorizer.get_feature_names())
    aux.columns=['word1']
    features_names= aux.merge(stem_match, how='left',left_on='word1',right_on='stem')['original']

In [31]:
CV_encoding=vectorizer.transform(textos_final);
pd.DataFrame(CV_encoding.todense(),columns=features_names).sum().sort_values(ascending=False)[0:20]

original
vacuna             50
cobrar             37
frontera           32
nino               32
tomar              29
marcha             28
cambio             28
responsabilidad    27
salta              26
chicos             26
punto              26
decia              25
familia            25
ingreso            25
habilitado         24
cruzan             24
necesita           23
escuchar           22
jujuy              22
deportes           22
dtype: int64

-----------------
### Term Frequency Inverse Document Frequency (TF-IDF)

In [32]:
Tfidf_encoding=TfidfTransformer().fit_transform(CV_encoding);
pd.DataFrame(Tfidf_encoding.todense(),columns=features_names,index=file).head()

original,aah,abalos,abarca,abasto,abdomen,abdominales,abertis,abierto,abogado,abrazo,...,wilson,yel,yivan,you,ysimple,ytener,yumbo,zelanda,zona,zumo
C5N 8-14 Step 1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.017935,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.00000,0.0,0.0,0.071741,0.0
C5N 8-14 Step 2,0.0,0.0,0.0,0.0,0.0,0.0,0.034229,0.019297,0.034229,0.029396,...,0.0,0.0,0.0,0.034229,0.00000,0.00000,0.0,0.0,0.000000,0.0
C5N 8-14 Step 3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.00000,0.0,0.0,0.025538,0.0
C5N 8-14 Step 4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.036319,0.000000,0.055325,...,0.0,0.0,0.0,0.000000,0.03221,0.03221,0.0,0.0,0.072637,0.0
C5N 8-14 Step 5,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.00000,0.00000,0.0,0.0,0.000000,0.0


-----------
### Tópicos

In [33]:
tfidf = pd.DataFrame(Tfidf_encoding.todense(),columns=features_names)
tfidf.index = file
tfidf= tfidf.T

topicos = []
for x in tfidf.columns:
    topico_x = ','.join(tfidf[x].sort_values(ascending=False)[0:8].index)
    topicos.append([x,topico_x])

topicos = pd.DataFrame(topicos,columns=['file','topico'])
pd.set_option('display.max_colwidth', 100)
topicos #Esto se puede usar para la nube, pero no prepondera palabras

,file,topico
0,C5N 8-14 Step 1,"colapsar,terapia,intensiva,pablo,posicion,morir,cambio,definiciones"
1,C5N 8-14 Step 2,"convocando,herramienta,fraude,calle,reforma,marcha,proyecto,movilizacion"
2,C5N 8-14 Step 3,"nino,avenida,carrera,educacion,comercio,utilizar,palermo,fecoba"
3,C5N 8-14 Step 4,"espana,nino,juguete,empieza,verano,ingreso,turistas,juan"
4,C5N 8-14 Step 5,"movilizaciones,republica,mental,muerte,tomar,caracter,mensaje,europa"
5,C5N 8-14 Step 6,"martin,san,vacuna,macri,marcha,incremento,aceptar,muerte"
6,TN 8-14 Step 1,"fuerzas,cobrar,frontera,salta,bolivia,ratito,detenidos,cruzan"
7,TN 8-14 Step 2,"cobrar,beneficios,funcionarios,frontera,jujuy,salta,fase,respiradores"
8,TN 8-14 Step 3,"vacuna,escuela,voto,concejales,chicos,cobrar,necesita,clases"
9,TN 8-14 Step 4,"discurso,sergio,fernando,exito,contradiccion,responsabilidad,decreto,confuso"


--------------
## Sentiment analysis

In [34]:
afinn = pd.read_csv("../Varios/lexico_afinn_es.csv",encoding='latin-1')
afinn.dropna(inplace=True)
afinn=afinn[['palabra','puntuacion']]
print("Palabras negativas")
display(afinn.sort_values(by='puntuacion').head(5))

print("Palabras positivas")
display(afinn.sort_values(by='puntuacion',ascending=False).head(5))

print("Palabras con poca tendencia")
afinn.loc[afinn['puntuacion'].between(-1,1),:].sample(5)

Palabras negativas


,palabra,puntuacion
584,coño,-5
1894,perras,-5
1757,Niggas,-5
1893,perra,-5
2346,tragona,-5


Palabras positivas


,palabra,puntuacion
1326,Hurra,5
910,encantados,5
283,asombroso,5
1072,excepcional,5
1608,magnífico,5


Palabras con poca tendencia


,palabra,puntuacion
68,adoptar,1
2421,veredictos,-1
206,anti,-1
1058,evitar,-1
291,asustadizo,-1


In [35]:
afinn_stem = []
for word in afinn['palabra']:
    afinn_stem.append(stemfraseesp_for_stopwords(word))

afinn_stem = afinn.join(pd.DataFrame(afinn_stem))
afinn_stem.columns = ['palabra','puntuacion','stem']
afinn_stem.drop_duplicates(subset='stem',inplace=True)
afinn_stem.head()

,palabra,puntuacion,stem
0,a bordo,1,abord
1,abandona,-2,abandon
4,abatido,-2,abat
6,aborrece,-3,aborrec
10,adoptar,1,adopt


In [36]:
file_stemCount = pd.DataFrame(CV_encoding.todense(),columns=vectorizer.get_feature_names(),index=file).T
file_stemCount.reset_index(inplace=True)
file_stemCount.rename(columns={'index':'word'}, inplace=True)

In [37]:
sentiment_analysis = file_stemCount.merge(afinn_stem,left_on='word',right_on='stem')
sentiment_analysis

for column in file.values:
    sentiment_analysis[column] = sentiment_analysis[column]*sentiment_analysis['puntuacion']

sentiment_analysis_final = sentiment_analysis[list(file.values)].sum().T
sentiment_analysis_final=pd.DataFrame(sentiment_analysis_final,columns=['sentiment'])
sentiment_analysis_final.reset_index(inplace=True)
sentiment_analysis_final.rename(columns={'index':'file'}, inplace=True)
sentiment_analysis_final.head()

,file,sentiment
0,C5N 8-14 Step 1,-58
1,C5N 8-14 Step 2,-8
2,C5N 8-14 Step 3,33
3,C5N 8-14 Step 4,12
4,C5N 8-14 Step 5,-52


In [38]:
final = topicos.merge(sentiment_analysis_final,on='file')
final[0:60]

,file,topico,sentiment
0,C5N 8-14 Step 1,"colapsar,terapia,intensiva,pablo,posicion,morir,cambio,definiciones",-58
1,C5N 8-14 Step 2,"convocando,herramienta,fraude,calle,reforma,marcha,proyecto,movilizacion",-8
2,C5N 8-14 Step 3,"nino,avenida,carrera,educacion,comercio,utilizar,palermo,fecoba",33
3,C5N 8-14 Step 4,"espana,nino,juguete,empieza,verano,ingreso,turistas,juan",12
4,C5N 8-14 Step 5,"movilizaciones,republica,mental,muerte,tomar,caracter,mensaje,europa",-52
5,C5N 8-14 Step 6,"martin,san,vacuna,macri,marcha,incremento,aceptar,muerte",46
6,TN 8-14 Step 1,"fuerzas,cobrar,frontera,salta,bolivia,ratito,detenidos,cruzan",-5
7,TN 8-14 Step 2,"cobrar,beneficios,funcionarios,frontera,jujuy,salta,fase,respiradores",-25
8,TN 8-14 Step 3,"vacuna,escuela,voto,concejales,chicos,cobrar,necesita,clases",-6
9,TN 8-14 Step 4,"discurso,sergio,fernando,exito,contradiccion,responsabilidad,decreto,confuso",2


In [39]:
final['canal'] = final['file'].apply(lambda x: x.split(' ')[0])

In [40]:
final.groupby(['canal'])['sentiment'].mean()

canal
C5N   -4.5
TN    -0.5
Name: sentiment, dtype: float64

In [41]:
final['macri'] = final['topico'].apply(lambda x: 1 if 'vacuna' in x else 0)


mask = final['macri'] == 1
final_macri = final.loc[mask,:]

print(final_macri.groupby(['canal'])['macri'].sum())
final_macri.groupby(['canal'])['sentiment'].mean()

canal
C5N    1
TN     1
Name: macri, dtype: int64


canal
C5N    46
TN     -6
Name: sentiment, dtype: int64

In [42]:
auxiliar= []
for x in final['topico']:
    asd = x.replace(',',' ')
    auxiliar.append(asd)

In [43]:
word_topico=[]

for topico_x in auxiliar:
    asd = word_tokenize(topico_x)
    for x in asd:
        word_topico.append(x)

In [44]:
pd.Series(word_topico).value_counts(ascending=False)[0:15]

nino         3
cobrar       3
salta        2
marcha       2
muerte       2
frontera     2
escuela      2
jujuy        2
vacuna       2
sergio       1
juegos       1
proyecto     1
caracter     1
morales      1
detenidos    1
dtype: int64